# SQLAlchemy Joins

## Setup

In [59]:
# Python SQL toolkit and Object Relational Mapper
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [60]:
rds_connection_string = "postgres:admin@localhost:5432/satellite"
#<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [61]:
pd.read_sql_query("select * from country where country_code in ('US','CIS')", con=engine).head()

,country,country_code,number_of_satellites
0,COMMONWEALTH OF INDEPENDENT STATES (FORMER USSR),CIS,1532
1,UNITED STATES,US,2739


In [62]:
engine.table_names()

['country', 'satellite_category', 'country_satellite']

In [63]:
#data_cnt = pd.read_sql_query('select * from country', con=engine).head()
data_cnt = engine.execute("select * from country where country_code in ('CIS', 'US')", con=engine)

In [64]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [65]:

datetime='2020-10-23'
data_symbol = {
    "country_code": "",
    "satellite_name": "", 
    "satellite_id": "", 
    "sat_intl_code": "", 
    "sat_launch_date": "", 
    "sat_period": ""

}
offsetVal=0







In [66]:


for record in data_cnt: 
    time.sleep(1)
    url = "https://www.n2yo.com/satellites/?c="+record['country_code']+"&t=country"
    browser.visit(url)
    time.sleep(200)
    html = browser.html 
    soup = bs(html, "html.parser") 
    data_array = [] 
    avg_temps = soup.find("table", class_="footable")     
    paragraphs = avg_temps.find_all('tr') 
    for para in paragraphs: 
        data_symbol={}
        try:
            data_symbol["country_code"] = record['country_code'] 
            data_symbol["satellite_name"] = para.find_all('td')[0].text 
            data_symbol["satellite_id"] = para.find_all('td')[1].text 
            data_symbol["sat_intl_code"] = para.find_all('td')[2].text
            tot_vol_str_data= para.find_all('td')[3].text            
            if (tot_vol_str_data.find('-0001') != -1):
                tot_vol_str_data=str(tot_vol_str_data).replace('-0001','2020')
            else:
                data_symbol["sat_launch_date"] = para.find_all('td')[3].text            
            if (para.find_all('td')[4].text != "-"):
                data_symbol["sat_period"] = para.find_all('td')[4].text
            else:
                data_symbol["sat_period"] = 0          
            data_array.append(data_symbol)
        except:
            pass
    data_array_df = pd.DataFrame(data_array)
    data_array_df.to_sql(name='country_satellite', con=engine, if_exists='append', index=False)




In [67]:
pd.read_sql_query('select * from country_satellite', con=engine).head()

,country_code,satellite_name,satellite_id,sat_intl_code,sat_launch_date,sat_period
0,ALG,ALCOMSAT 1,43039,2017-078A,2017-12-10,1436.1
1,ALG,ALSAT 1N,41789,2016-059G,2016-09-26,98.3
2,ALG,ALSAT 2B,41786,2016-059D,2016-09-26,98.2
3,ALG,ALSAT 1B,41785,2016-059C,2016-09-26,98.4
4,ALG,ALSAT 2A,36798,2010-035D,2010-07-12,98.2


In [49]:
engine.close()

AttributeError: 'Engine' object has no attribute 'close'